In [1]:
import torch
import sys
sys.path.insert(0, "../..")
from src.data import data_tools, make_dataset
from pathlib import Path
from loguru import logger

In [2]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [3]:
data_dir = "../../data/raw"
image_folder = make_dataset.get_flowers(data_dir)
dataloader = data_tools.Dataloader(path=image_folder, split=0.8)

2022-05-19 13:00:20.825 | INFO     | src.data.make_dataset:get_flowers:74 - Data is downloaded to ../../data/raw/datasets/flower_photos.


In [4]:
len(dataloader)

3670

In [5]:
dataloader.class_dict

{'roses': 0, 'sunflowers': 1, 'daisy': 2, 'dandelion': 3, 'tulips': 4}

In [6]:
from torchvision import transforms
preprocess = transforms.Compose([
    transforms.ToTensor(),
])

In [7]:
config = {
    "batch_size" : 32,
    "image_size" : (224,224),
    "channels" : 3,
    "shuffle" : True,
    "channel_first" : True,
    "transforms" : preprocess
}

train_datastream = dataloader.data_generator(
    mode="train",
    **config
)

test_datastream = dataloader.data_generator(
    mode="test",
    **config
)

In [8]:
X, y = next(train_datastream)
X.shape, y.shape

(torch.Size([32, 3, 224, 224]), torch.Size([32]))

In [9]:
X.max(), X.min(), X.mean(), X.std()

(tensor(1.), tensor(0.), tensor(0.4399), tensor(0.2929))

In [10]:
resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Using cache found in /Users/rgrouls/.cache/torch/hub/pytorch_vision_v0.10.0


In [11]:
yhat = resnet(X)

In [12]:
yhat.shape

torch.Size([32, 1000])

In [13]:
for name, param in resnet.named_parameters():
    if ("bn" not in name):
        param.requires_grad = False

In [14]:
import torch.nn as nn
resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 500),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(500, 5)
)

In [15]:
yhat = resnet(X)

In [16]:
optimizer = torch.optim.Adam(resnet.parameters(), lr=1e-3)

In [17]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer
)

In [18]:
loss_fn = nn.CrossEntropyLoss()
loss_fn(yhat, y)

tensor(1.6380, grad_fn=<NllLossBackward0>)

In [19]:
(len(dataloader) * 0.8) // 32

91.0

In [ ]:
from src.models import train_model
resnet = train_model.trainloop(
    epochs = 10,
    model = resnet,
    optimizer= torch.optim.Adam,
    learning_rate=0.1,
    loss_fn=nn.CrossEntropyLoss(),
    train_dataloader=train_datastream,
    test_dataloader=test_datastream,
    log_dir="../../models/resnet",
    eval_steps=15,
    train_steps=25,
    patience=2,
    factor=0.5,
)